In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/youtoxic_english_1000.csv")

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


In [ ]:
# 1. Revisar columnas con solo ceros o sin representación significativa
etiquetas = [
    "IsToxic", "IsAbusive", "IsThreat", "IsProvocative", "IsObscene",
    "IsHatespeech", "IsRacist", "IsNationalist", "IsSexist", "IsHomophobic",
    "IsReligiousHate", "IsRadicalism"
]

# Mostrar cuántas veces aparece "True" en cada etiqueta
print("Etiquetas activas:")
print(df[etiquetas].sum().sort_values())

# 2. Eliminar columnas sin representación (sum == 0 o sum == 1)
etiquetas_utiles = [col for col in etiquetas if df[col].sum() > 1]
print("\nEtiquetas que conservaremos:", etiquetas_utiles)

# 3. Eliminar columnas irrelevantes para el modelado
columnas_irrelevantes = ["CommentId", "VideoId"]
df.drop(columns=columnas_irrelevantes, inplace=True)

# 4. (Opcional) Eliminar duplicados por texto
df.drop_duplicates(subset="Text", inplace=True)

# 5. Dejar el DataFrame con solo las columnas útiles
columnas_utiles = ["Text"] + etiquetas_utiles
df = df[columnas_utiles]

# 6. Confirmar
print(f"\nDataset limpio → Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.head()


### Limpieza previa del dataset

Antes de aplicar el preprocesamiento textual, revisamos las columnas del dataset:

- Eliminamos etiquetas que **no tienen representación suficiente** (por ejemplo, `IsHomophobic`, `IsRadicalism`) ya que no aportarían valor al modelo.
- Quitamos columnas irrelevantes como `CommentId` y `VideoId`.
- Eliminamos posibles duplicados exactos en los comentarios (`Text`).

Esto nos permite trabajar sobre un dataset más limpio, centrado en los comentarios y en las etiquetas que tienen información útil. De este modo, evitamos introducir ruido o clases vacías en el modelo.
